# S3 Presigned URLs: overview

_"Pre-signed URLs allow you to give your users access to a specific object
in your bucket without requiring them to have AWS security credentials or permissions."_

There are two types of pre-signed urls that can be generated:
1. Presigned url (limited functionality)
2. Presigned post url (extended functionality)

For generating pre-signed and post pre-signed urls with Python,
we will use Boto3 package. You can install it simply with pip by running:

In [ ]:
!pip install boto3
!pip install COMPREDICT-SDK-AI

## Initialize COMPREDICT Client

In [ ]:
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
from environ import Env
import os

from compredict.client import api

env = Env()
env.read_env()

In [ ]:
token = env("COMPREDICT_AI_CORE_KEY")
fail_on_error = env("COMPREDICT_AI_CORE_FAIL_ON_ERROR", False)

client = api.get_instance(token=token)
client.fail_on_error(option=fail_on_error)

## Generate presigned URL

In order to generate a pre-signed URL, first we need to set the credentials of S3 and some information about the uploaded file:

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = env("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = env("AWS_SECRET_ACCESS_KEY")
region_name = env("REGION")
file_name = "path/to/file/in/s3/my_file.json"
bucket_name = env("BUCKET")

Let's implement function for creating pre-signed URL. Notice the usage of boto3 and it's client. This function return
 simple string with generated url.

In [ ]:
def create_get_presigned_url(bucket_name, object_name, expiration=3600):
    """Generate a presigned URL S3 POST request to upload a file

    :param bucket_name: string
    :param object_name: string
    :return: Dictionary with the following keys:
        url: URL to post to
        fields: Dictionary of form fields and values to submit with the POST
    :return: None if error.
    """
    # Generate a pre-signed S3 POST URL
    s3_client = boto3.client('s3', config=Config(signature_version='s3v4'), region_name=region_name)
    try:
        response = s3_client.generate_presigned_url('put_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name,
                                                            },
                                                    ExpiresIn=expiration)
    except ClientError as e:
        return None

    return response

Now, after setting env variables, specifying filename and bucket, pre-signed url can be generated with the function
implemented above.

In [ ]:
url = create_get_presigned_url(bucket_name, file_name)

Pre-signed URL that we generated, can be used now as callback_url.

In order to run algorithm with pre-signed URL, simply add generated url value to `callback_url`arg.

In [ ]:
task = client.run_algorithm('an-algorithm-id', data, evaluate=False, callback_url=generated_url)

## Presigned Post URL

For extended functionality, post pre-signed url can be generated. Presigned Post URL allows you to define:

- ACL fields.
- Meta data of files
- Content Type of the uploaded file.

Now, let's implement a **function for generating post pre-signed url**. Notice, that this function, returns
dictionary with not only generated url, but also *fields* dictionary.
We will need it later when running an algorithm.

In [ ]:
def create_presigned_post(bucket_name, object_name, fields=None, conditions=None, expiration=3600):
    # Generate a presigned S3 POST URL
    s3_client = boto3.client('s3', config=Config(signature_version='s3v4'), region_name=region_name)
    try:
        response = s3_client.generate_presigned_post(
            Bucket=bucket_name,
            Key=object_name,
            Fields=fields,
            Conditions=conditions,
            ExpiresIn=expiration
        )
    except ClientError as e:
        print(str(e))
        return None

    pre_signed_url = response['url']
    fields = response['fields']

    return pre_signed_url, fields

In [ ]:
# Adding the ACL and some meta information of the file.
fields = {"ACL": "bucket-owner-full-control", "x-amz-meta-a-key": "a-value"}
conditions = [{"ACL": "bucket-owner-full-control"}, {"x-amz-meta-a-key": "a-value"}]

url, s3_fields = create_presigned_post(bucket_name, file_name, fields=fields, conditions=conditions)

Now, we can use our generated *pre_signed_url* and *fields* as callback_url and callback_param arguments when running
algorithms. In case of post pre-signed url, value for callback_param is required.

Let's use `fields` to create callback_param dictionary. We have to assign `fields` to `s3_fields` key.

In [ ]:
callback_param = {'s3_fields': fields}

# In case of multiple callbacks where first callback is the s3 generated url
# callback_param = [{'s3_fields': fields}, {}]

Now we can run algorithm with `callback_url` as our post pre-signed url and `callback_param` as dictionary created above.

In [ ]:
task = client.run_algorithm('an-algorithm-id', data, evaluate=False,
                            callback_url=generated_url, callback_param=callback_param)

If `s3_fields` wasn't given in callback_param, then AI Core will assume this is a presigned url and will send the results as PUT to S3 and will fail.

## Uploaded File

By default The uploaded file to S3 will be json file with the following format:

~~~json
{
    "job_id": str,
    "status": "Finished",
    "callback_param": dict,
    "success": bool,
    "results": dict
}
~~~

Where results is a dictionary containing the predictions and evaluations if given.

For further documentation on boto3 and presigned urls, please visit this [tutorial](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.generate_presigned_post). 